# Using Reinforcement Learning to solve a Smartcab problem
---
**Author**: Marko Bajec

**Last update**: 12.3.2019

**Description**: this tutorial demonstrates how to use reinforcement learning (RL) for solving a problem of autonomous taxi car (Smartcab). The goal of the Smartcab is to pick up a passenger from a start location and drive him to the final location. The position of the Smartcab and start/final locations are all predefined for each taxi job. The algorithm has to learn three things: a) how to pick up a passenger (the taxi has to be at the location of the passenger), b) how to drive the passenger to the final location by using an optimal route, and c) how to drop off the passenger (again, this is only possible if the taxi is at the final location and the passenger is in the car). 

The tutorial is based on [Reinforcement Q-Learning from Scratch in Python with OpenAI Gym](https://www.learndatasci.com/tutorials/reinforcement-q-learning-scratch-python-openai-gym/).

For gym documentation see [here](https://gym.openai.com/docs/).

---

In [ ]:
!pip3 install cmake 'gym[atari]' scipy

## Modeling the Smartcab problem

In RL, the problem is modeled by specifying **rewards**, **states**, and **actions**.

### Rewards
Since the agent (the imaginary driver) is reward-motivated and is going to learn how to control the cab by trial experiences in the environment, we need to decide the rewards and/or penalties and their magnitude accordingly. Here are a few points to consider:

* The agent should receive a high positive reward for a successful dropoff because this behavior is highly desired, let's say <code>20</code>,
* The agent should be penalized if it tries to drop off a passenger in wrong locations, let it be <code>-10</code>
* The agent should get a slight negative reward for not making it to the destination after every time-step. "Slight" negative because we would prefer our agent to reach late instead of making wrong moves trying to reach to the destination as fast as possible, e.g. <code>-1</code>.

### State Space
In RL, the agent encounters a state, and then takes action according to the state it's in. The **State Space** is the set of all possible situations our taxi could inhabit. The state should contain useful information the agent needs to make the right action.

Let's say we have a training area for our Smartcab where we are teaching it to transport people in a parking lot to four different locations (<code>R</code>, <code>G</code>, <code>Y</code>, <code>B</code>):

![Representation of the taxi problem environment](https://storage.googleapis.com/lds-media/images/Reinforcement_Learning_Taxi_Env.width-1200.png).

### Action Space
The agent encounters one of the 500 states and it takes an action. The action in our case can be to move in a direction or decide to pickup/dropoff a passenger. In other words, we have six possible actions:
* <code>south</code>,
* <code>north</code>,
* <code>east</code>,
* <code>west</code>,
* <code>pickup</code>,
* <code>dropoff</code>.

You'll notice in the illustration above, that the taxi cannot perform certain actions in certain states due to walls. In environment's code, we will simply provide a <code>-1</code> penalty for every wall hit and the taxi won't move anywhere. This will just rack up penalties causing the taxi to consider going around the wall.

## Setting the environment in Python
To setup the environment, we import *gym* library and load "Taxi-v2" *environment*. This environment has already been prepared. It consists of:
* 500 states (5x5 possible taxi positions, 5 positions of a passanger - including one in taxi, and 4 posible drop off locations. Altogether 5 x 5 x 5 x 4 = 500),
* 6 actions (west, east, north, south, drop off, pick up).

Note: if you haven't done so yet, install the *gym* library using <code>!pip3 install cmake 'gym[atari]' scipy</code>. Then try to import the library and render the space.   

In [ ]:
import gym

env = gym.make("Taxi-v2").env

env.render()

In [ ]:
env.reset() # reset environment to a new, random state
env.render()

print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

To setup a particular situation, environment method <code>encode</code> can be used. Remember that our state is described with four elements:
* taxi x position,
* taxi y position,
* passanger index, and
* destination index.

In [ ]:
state = env.encode(3, 1, 2, 0) # (taxi row, taxi column, passenger index, destination index)
print("State:", state)

env.s = state
env.render()

### The Reward Table
When the Taxi environment is created, there is an initial **Reward table** that's also created, called `P`. We can think of it like a matrix that has the number of states as rows and number of actions as columns, i.e. a $states × actions$ matrix.

<p>To output the rewards for a particluar state and its actions, use <code>env.P[state]</code>.</p> 
<p>Each state, action pair is described with the following structure: <code>action: [(probability, nextstate, reward, done)].</code></p>

In [ ]:
env.P[328]

A few things to note:
* The <code>0-5</code> corresponds to the actions (*south*, *north*, *east*, *west*, *pickup*, *dropoff*) the taxi can perform at our current state in the illustration.
* In this env, <code>probability</code> is always 1.0.
* The <code>nextstate</code> is the state we would be in if we take the action at this index of the dict
* All the movement actions have a -1 reward and the pickup/dropoff actions have -10 reward in this particular state. If we are in a state where the taxi has a passenger and is on top of the right destination, we would see a reward of 20 at the dropoff action (5)
* <code>done</code> is used to tell us when we have successfully dropped off a passenger in the right location. Each successfull dropoff is the end of an episode.

<p>Note that if our agent chose to explore action three (3) in this state it would be going West into a wall. The source code has made it impossible to actually move the taxi across a wall, so if the taxi chooses that action, it will just keep accruing -1 penalties (staying at the same position), which affects the **long-term reward**.</p>

## Solving the taxi problem using brute force  
We can try to solve the taxi problem by randomly seleting actions. We finish when the passanger is picked up and droped off at the final destination. It is clear from the example, that such an approach is useless - we make many wrong steps and receive a lot of penalties.  

To randomly select an action, the method <code>sample</code> is used: <code>action = env.action_space.sample()</code>.

In [ ]:
env.s = 328  # set environment to illustration's state

epochs = 0
penalties, reward = 0, 0

frames = [] # for animation

done = False

while not done:
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)

    if reward == -10:
        penalties += 1
    
    # Put each rendered frame into dict for animation
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    epochs += 1
    
    
print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

### Simulating the environment
To see the simulation of how the system was choosing actions, we just print the dictionary of frames. 

In [ ]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'].getvalue())
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.1)
        
print_frames(frames)

What we can see is that our agent takes thousands of timesteps and makes lots of wrong drop offs to deliver just one passenger to the right destination. This is because we aren't *learning* from past experience. We can run this over and over, and it will never optimize. The agent has no memory of which action was best for each state, which is exactly what **Reinforcement Learning** will do for us.

## Using Reinforcement Learning to solve the taxi problem

We are going to use a simple RL algorithm called **Q-learning** which will give our agent some memory.

### Intro to Q-learning
Essentially, Q-learning lets the agent use the environment's rewards to learn, over time, the best action to take in a given state.

In our Taxi environment, the reward table, <code>P</code> can help us to learn from past experience. This is done by taking an action in the current state and then updating a **Q-value** to remember if that action was beneficial. Q-values are remembered/updated every time an action is made in a particular state. Notice that Q-values pertain to a <code>state, action</code> pairs rather than to <code>states</code>. They are stored in a **Q-table**. 

A Q-value for a particular <code>state-action</code> combination is representative of the "quality" of an action taken from that state. Better Q-values imply better chances of getting greater rewards.

For example, if the taxi is faced with a state that includes a passenger at its current location, it is highly likely that the Q-value for <code>pickup</code> is higher when compared to other actions, like <code>dropoff</code> or <code>north</code>.

Q-values are initialized to an arbitrary value, and as the agent exposes itself to the environment and receives different rewards by executing different actions, the Q-values are updated using the following equation:

$$Q(state,action)←(1−α)*Q(state,action)+α*(reward + γ*\underset{a}{\operatorname{max}}Q(next state,a))$$

where:

- $α$ (alpha) is the learning rate $(0<α≤1)$. It tells the extent to which our Q-values are being updated in every iteration.
- $γ$ (gamma) is the discount factor $(0≤γ≤1)$. It determines how much importance we want to give to future rewards. A high value for the discount factor (close to 1) captures the long-term effective award, whereas, a discount factor of 0 makes our agent consider only immediate reward, hence making it $greedy$.

### Exploiting learned values

After enough random exploration of actions, the Q-values tend to **converge** serving our agent as an action-value function which it can **exploit** to pick the most optimal action from a given state.

There's a tradeoff between **exploration** (choosing a random action) and **exploitation** (choosing actions based on already learned Q-values). We want to prevent the action from always taking the same route, and possibly overfitting, so we'll be introducing another parameter called $ϵ$ "epsilon" to cater to this during training.

Instead of just selecting the best learned Q-value action, we'll sometimes favor exploring the action space further. Lower epsilon value results in **episodes** with more penalties (on average) which is obvious because we are exploring and making random decisions.

### Training the agent

First, we'll initialize the Q-table to a 500×6 matrix of zeros:


In [ ]:
import numpy as np
q_table = np.zeros([env.observation_space.n, env.action_space.n])

We can now create the training algorithm that will update the <code>Q-table</code> as the agent explores the environment over thousands of episodes.

In the first part of while not done, we decide whether to pick a random action or to exploit the already computed Q-values. This is done simply by using the **epsilon value** and comparing it to the <code>random.uniform(0, 1)</code> function, which returns an arbitrary number between 0 and 1.

We execute the chosen action in the environment to obtain the <code>next_state</code> and the <code>reward</code> from performing the action. After that, we calculate the <code>maximum Q-value</code> for the actions corresponding to the <code>next_state</code>, and with that, we can easily update our Q-value to the <code>new_q_value</code>.

In [ ]:
%%time
"""Training the agent"""

import random
from IPython.display import clear_output

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, 100001):
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, done, info = env.step(action) 
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")

Now that the Q-table has been established over 100,000 episodes, let's see what the Q-values are at our illustration's state:

In [ ]:
q_table[328]

Remember that Q-table tells the rewords for each of the possible actions (*west*, *east*, *north*, *south*, *drop off*, and *pick up*). The max Q-value is for "east", so it looks like Q-learning has effectively learned the best action to take in our illustration's state!

### Evaluation
Now let's try how good is our model (Q-table). We will run 100 episodes and observe how many mistakes the Smartcab will do (remember that for each wrong move, i.e. wrong pick up, wrong drop off or wrong move - move to a wall, we get a penalty). What we expect is that since we use the learned model, the penalties should only rarely occur if at all.

In [ ]:
"""Evaluate agent's performance after Q-learning"""

total_epochs, total_penalties = 0, 0
episodes = 100

for _ in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)

        if reward == -10:
            penalties += 1

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")